In [1]:
%matplotlib inline
import os
import re
import datetime as dt
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from args_tools_jupyter import args, createfolder
pd.set_option('precision', 5)

In [2]:
ty_list = pd.read_csv(args.ty_list)
ty_list['Time of canceling'] = pd.to_datetime(ty_list['Time of canceling'])
ty_list['Time of issuing'] = pd.to_datetime(ty_list['Time of issuing'])
ty_list

,En name,Ch name,Time of issuing,Time of canceling
0,SAOLA,蘇拉,2012-07-31 21:00:00,2012-08-03 14:00:00
1,SOULIK,蘇力,2013-07-12 12:00:00,2013-07-13 23:00:00
2,TRAMI,潭美,2013-08-20 20:00:00,2013-08-22 08:00:00
3,KONG-REY,康芮,2013-08-28 11:00:00,2013-08-29 17:00:00
4,USAGI,天兔,2013-09-20 08:00:00,2013-09-22 08:00:00
5,FITOW,菲特,2013-10-05 14:00:00,2013-10-07 08:00:00
6,MATMO,麥德姆,2014-07-22 02:00:00,2014-07-23 23:00:00
7,CHAN-HOM,昌鴻,2015-07-09 20:00:00,2015-07-10 23:00:00
8,SOUDELOR,蘇迪勒,2015-08-07 09:00:00,2015-08-09 20:00:00
9,DUJUAN,杜鵑,2015-09-28 06:00:00,2015-09-29 17:00:00


In [206]:
createfolder(args.ty_info_wrangled_data_folder)
for j in range(len(ty_list)):
    year = str(ty_list.iloc[j,-1].year)
    tyname = ty_list.iloc[j,0]
    print(tyname)
    ty_track_path = os.path.join(args.weather_raw_data_folder, year+'.Data',  year+'_TrackData',  year+'.'+tyname+'.Track.txt')
    ty_track = pd.read_csv(ty_track_path, sep='\s+', skiprows=0)
    ty_track = ty_track.assign(Time=ty_track.Typhname.astype(str)+' '+ty_track.Date.astype(str))
    ty_track = ty_track.drop(columns=['Typhname', 'Date']).reindex(columns=['Time', 'Lat', 'Lon','Press','Wind','Gust','7Dir','10Dir','Warn'])
    ty_track.Time = pd.to_datetime(ty_track.Time) + dt.timedelta(hours=8)
    ty_track.set_index('Time',inplace=True)
    ty_track[ty_track == -99.0] = 0

    timestep = ty_track.index[-1]-ty_track.index[0]
    timestep = timestep.days*24*6+int(timestep.seconds/3600)
    index = pd.Index([ty_track.index[0] + dt.timedelta(hours=x) for x in range(timestep+1)], name='Time')
    new_ty_track = pd.DataFrame([],index=index,columns=ty_track.columns)

    for i in range(len(ty_track)-1):
        timestep = int((ty_track.index[i+1]-ty_track.index[i]).seconds/600)
        for k in range(timestep+1):
            new_ty_track.loc[ty_track.index[i]+dt.timedelta(hours=k/6),:] = ty_track.iloc[i] + k*(ty_track.iloc[i+1]-ty_track.iloc[i])/timestep
    
    output_timestep = ty_list.iloc[j,-1]-ty_list.iloc[j,-2]
    output_timestep = output_timestep.days*24*6+int(output_timestep.seconds/600)
    output_index = pd.Index([ty_list.iloc[j,-2] + dt.timedelta(hours=x/6) for x in range(output_timestep+1)], name='Time')
    new_ty_track = new_ty_track.reindex(index=output_index)
    outputname = os.path.join(args.ty_info_wrangled_data_folder, year+'.'+tyname+'.csv')
    new_ty_track.to_csv(outputname)

SAOLA
SOULIK
TRAMI
KONG-REY
USAGI
FITOW
MATMO
CHAN-HOM
SOUDELOR
DUJUAN
MALAKAS
MEGI
MARIA


In [47]:
for idx, file in enumerate(sorted(os.listdir(args.ty_info_wrangled_data_folder))):
    data = pd.read_csv(os.path.join(args.ty_info_wrangled_data_folder,file),index_col='Time')
    if idx == 0:
        tmp_max = np.max(data)
        tmp_min = np.min(data)
    else:
        if (np.max(data) > tmp_max).any():
            tmp_max[np.max(data) > tmp_max] = np.max(data)[np.max(data) > tmp_max]
        if (np.min(data) < tmp_min).any():
            tmp_min[np.min(data) < tmp_min] = np.min(data)[np.min(data) < tmp_min]

In [51]:
overall_df = pd.DataFrame({'max_value':tmp_max, 'min_value':tmp_min})
overall_df.to_csv(os.path.join(args.ty_info_folder,'overall.csv'))

In [50]:
args.ty_info_folder

'/home/jack/ssd/01_ty_research/03_ty_info'